In [71]:
from openpyxl import Workbook
wb = Workbook()

In [72]:
wb = Workbook()
ws1 = wb.create_sheet('Giełda')
ws2 = wb.create_sheet('Linki')
ws3 = wb.create_sheet('Filmweb')

In [73]:
import requests
from bs4 import BeautifulSoup
from random import choice
from string import ascii_lowercase
import re

In [75]:
def generate_String(y):
    
    return "".join(choice(ascii_lowercase) for i in range(y))
codes = [generate_String(3) for _ in range(3)]
s = requests.Session()
valid_codes = []



for code in codes:
   
    params = {"s": code}
    response = requests.get("http://stooq.pl/q/", params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    if len(soup.body.findAll('span', id='c2')) != 0:
        link_content = soup.find('font',id='f16').find('a',href=True)
        if link_content.text == 'symbol waloru':
            print("dla",code,"nie znaleźiono odpowiadającego kodu.")
        else:
            valid_code = link_content['href'][5:]
            valid_codes.append(valid_code)
            print("dla",code,f'Znaleziono kod podobny: {valid_code}')
    else:
        valid_codes.append(code)

        
ws1.cell(row=1, column=1).value = 'Kod'
ws1.cell(row=1, column=2).value = 'Kurs'
ws1.cell(row=1, column=3).value = 'Zmiana procentowa'
ws1.cell(row=1, column=4).value = 'Transakcje'
row = 2


for code in valid_codes:
    params = {"s": code}
    response = requests.get("http://stooq.pl/q/", params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    price = soup.find(text="Kurs").parent.find("span").text
    percent_change = soup.find(text="Zmiana").parent.findAll("span")[1].text
    transactions = soup.find(text="Transakcje").parent.find("span").text
    print(code,f"kusrs:{price}, zmiana procntowa:{percent_change},trransakcje:{transactions}")
    ws1.cell(row=row, column=1).value = code
    ws1.cell(row=row, column=2).value = price if price else 'Brak danych'
    ws1.cell(row=row, column=3).value =  percent_change if percent_change else 'Brak danych'
    ws1.cell(row=row, column=4).value =  transactions if transactions else 'Brak danych'
    row += 1

dla ykj nie znaleźiono odpowiadającego kodu.
dla nsa Znaleziono kod podobny: nsa.us
dla zru Znaleziono kod podobny: zru20.f
nsa.us kusrs:33.8400, zmiana procntowa:(-0.97%),trransakcje:1 544
zru20.f kusrs:11.880, zmiana procntowa:(-0.25%),trransakcje:


In [76]:
url = "https://jbzdy.cc/"
req = requests.get(url)
links = set()
soup = BeautifulSoup(req.content, 'html.parser')

for atag in soup.find_all("a"):
    link = atag.get('href')
    links.add(link)

row = 1
for link in links:
    ws2.cell(row,column=1).value = link
    row +=1

In [79]:
import bs4 as bs
import urllib.request

source_gl = urllib.request.urlopen('https://www.filmweb.pl/Gladiator').read()
soup_gl = bs.BeautifulSoup(source_gl,'lxml')

filmweb = {}
filmweb['premiera:'] = soup_gl.find("th", text="premiera:").find_next_sibling("td").text
filmweb['reżyseria:'] = soup_gl.find("th", text="reżyseria:").find_next_sibling("td").text
filmweb['boxoffice:']  = soup_gl.find("th", text="boxoffice:").find_next_sibling("td").text 
filmweb['ocena:'] = soup_gl.find("span", itemprop="ratingValue").text

row = 1
for key, value in filmweb.items():
    ws3.cell(row,column=1).value = key
    ws3.cell(row,column=2).value = value
    row += 1


In [81]:
wb.save('lab4.xlsx')